<a href="https://colab.research.google.com/github/tousif47/Mini-LLM/blob/main/Scratch_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

print("Done")

Done


In [27]:
data_text = open("/content/drive/MyDrive/Practice Playing with code/health.txt", "r")

print(data_text.read())
print("\n\nNumber of characters in text file :", len(open("/content/drive/MyDrive/Practice Playing with code/health.txt", "r").read()))

Being healthy and fit in simple terms means taking good care of the body. We should remember that a healthy mind resides only in a healthy body. Good health of both mind and body helps one maintain the required energy level to achieve success in life. All of us must strive to achieve wholesome health. Protecting your body from the intake of harmful substances, doing regular exercises, having proper food and sleep are some of the important instances that define a healthy lifestyle. Aldo being fit allows us to perform our activities without being lethargic, restless or tired. A healthy and fit person is capable of living the life to the fullest, without any major medical or physical issues. Being healthy is not only related to the physical well-being of a person, it also involves the mental stability or the internal peace of a person.

Generally, a healthy diet consists of taking a proper and healthy food which includes eating green and fresh vegetables, fruits, having milk, eggs, minera

In [ ]:
# Preprocessing the text data
# Basic text preprocessing because a simple dataset with minimal noise has been selected to simplify the code

